In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/2.make.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Year'] = df_input['Year'].astype(int)
df_input['Make No'] = df_input['Make No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：2361



,Year,Make No,Make
0,2025,1,Cadillac
1,2025,2,INFINITI
2,2025,3,Toyota
3,2024,1,Acura
4,2024,2,Alfa Romeo
...,...,...,...
2356,1959,2,Mercury
2357,1959,3,Plymouth
2358,1958,1,Mercury
2359,1958,2,Plymouth


In [3]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 Year           2025
 Make No           1
 Make       Cadillac
 Name: 0, dtype: object)

In [4]:
import time

data = {'action': 'bf_ajax_hook',
        'type': 'model',
        'c_year': crawler_series['Year'],
        'c_make': crawler_series['Make']}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = 'https://www.rainx.com/'

        resp = requests.post('https://www.rainx.com/wp-admin/admin-ajax.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            if html.xpath('//option/text()')[0].strip() == 'Model':
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

print(soup.prettify())

1
<html>
 <body>
  <option value="">
   Model
  </option>
  <option value="CT5">
   CT5
  </option>
  0
 </body>
</html>



In [5]:
list_model = [text for text in html.xpath('//option/text()')[1:]]

list_model

['CT5']

In [6]:
list_dict_correct.extend([{'Year': crawler_series['Year'],
                           'Make No': crawler_series['Make No'],
                           'Make': crawler_series['Make'],
                           'Model No': i + 1,
                           'Model': model} for i, model in enumerate(list_model)])

list_dict_correct

[{'Year': 2025,
  'Make No': 1,
  'Make': 'Cadillac',
  'Model No': 1,
  'Model': 'CT5'}]

In [7]:
crawler_status = True

crawler_status

True

In [8]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[Year                           2025
 Make No                           1
 Make                       Cadillac
 Check_Url    https://www.rainx.com/
 Name: 0, dtype: object]

In [9]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Year', 'Make No', 'Model No'],
                                                             ascending=[False, True, True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_model.xlsx', index=False)

df_correct

,Year,Make No,Make,Model No,Model
0,2025,1,Cadillac,1,CT5


In [10]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Year', 'Make No'],
                                                           ascending=[False, True],
                                                           ignore_index=True)
    df_error.to_excel('./test_model_error.xlsx', index=False)

df_error

,Year,Make No,Make,Check_Url
0,2025,1,Cadillac,https://www.rainx.com/
